In [1]:
import pandas as pd 
import numpy as np 
from matplotlib import pyplot as plt 

# import data
divvy_2019 = pd.read_csv('cleaned_data/divvy_2019_trips.csv')
divvy_2020 = pd.read_csv('cleaned_data/divvy_2020_trips.csv')


In [2]:
# Clean Up Missing Data on the Loop (To be implemented in etl scipt)
# Due to Formatting of Text

divvy_2019.loc[divvy_2019['from_community_name']=='(The) Loop[11]',['from_community_name']] = "Loop"
divvy_2019.loc[divvy_2019['from_community_name']=='Loop',['from_chicago_area']] = "Central"
divvy_2019.loc[divvy_2019['to_community_name']=='(The) Loop[11]',['to_community_name']] = "Loop"
divvy_2019.loc[divvy_2019['to_community_name']=='Loop',['to_chicago_area']] = "Central"

In [3]:
divvy_2020.loc[divvy_2020['from_community_name']=='(The) Loop[11]',['from_community_name']] = "Loop"
divvy_2020.loc[divvy_2020['from_community_name']=='Loop',['from_chicago_area']] = "Central"
divvy_2020.loc[divvy_2020['to_community_name']=='(The) Loop[11]',['to_community_name']] = "Loop"
divvy_2020.loc[divvy_2020['to_community_name']=='Loop',['to_chicago_area']] = "Central"

In [4]:
# Check for Missing Values
divvy_2020[divvy_2020['from_chicago_area'].isna()]

,started_at,ride_id,rideable_type,started_at.1,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,from_community_area,from_community_name,from_chicago_area,to_community_area,to_community_name,to_chicago_area


In [14]:
# create map to confirm station placements / accuracy of Community Area Numbers
import requests
import rtree
import geopandas as gpd
from shapely.geometry import Point, Polygon
from shapely import wkt


# Community Area Map
chicago = gpd.read_file("divvy_data/Boundaries - Community Areas", crs = {'init': 'epsg:4326'})

# Divvy Stations
divvy_stations_request = requests.get('https://data.cityofchicago.org/resource/bbyy-e7gq.json')
divvy_stations_json = pd.read_json(divvy_stations_request.text)
divvy_stations = pd.DataFrame(divvy_stations_json)
divvy_stations = gpd.GeoDataFrame(divvy_stations, geometry=gpd.points_from_xy(divvy_stations.longitude, divvy_stations.latitude))

In [15]:
def find_polygon(coordinate):
    dex =  chicago[chicago['geometry'].contains(coordinate)].index.values[0]
    return chicago['area_num_1'][dex]

In [22]:
divvy_stations = divvy_stations[~divvy_stations[':@computed_region_vrxf_vc4k'].isna()]

In [23]:
divvy_stations['fix_area'] = divvy_stations['geometry'].apply(lambda x: find_polygon(x))

<ipython-input-23-0e094b3a8e39>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  divvy_stations['fix_area'] = divvy_stations['geometry'].apply(lambda x: find_polygon(x))


In [24]:
divvy_stations

,id,station_name,total_docks,docks_in_service,status,latitude,longitude,location,:@computed_region_awaf_s7ux,:@computed_region_6mkv_f3dw,:@computed_region_vrxf_vc4k,:@computed_region_bdys_3d7i,:@computed_region_43wa_7qmu,geometry,fix_area
0,2,Buckingham Fountain,39,39,In Service,41.876511,-87.620548,"{'latitude': '41.87651122881695', 'longitude':...",22.0,14913,38.0,367.0,36.0,POINT (-87.62055 41.87651),32
1,3,Shedd Aquarium,55,55,In Service,41.867226,-87.615355,"{'latitude': '41.86722595682', 'longitude': '-...",48.0,14913,34.0,374.0,10.0,POINT (-87.61536 41.86723),33
2,4,Burnham Harbor,23,23,In Service,41.856268,-87.613348,"{'latitude': '41.856268', 'longitude': '-87.61...",48.0,14913,34.0,374.0,10.0,POINT (-87.61335 41.85627),33
3,5,State St & Harrison St,23,23,In Service,41.874053,-87.627716,"{'latitude': '41.874053', 'longitude': '-87.62...",48.0,14913,38.0,12.0,10.0,POINT (-87.62772 41.87405),32
4,6,Dusable Harbor,39,39,In Service,41.886976,-87.612813,"{'latitude': '41.886976', 'longitude': '-87.61...",22.0,21182,38.0,159.0,36.0,POINT (-87.61281 41.88698),32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664,1436495096608724736,N Clark St & W Elm St,4,4,In Service,41.902901,-87.631282,"{'latitude': '41.902901', 'longitude': '-87.63...",22.0,14926,37.0,310.0,11.0,POINT (-87.63128 41.90290),8
665,1436495100903692032,W Oakdale Ave & N Broadway,6,6,In Service,41.935581,-87.644042,"{'latitude': '41.935581', 'longitude': '-87.64...",38.0,4449,57.0,102.0,25.0,POINT (-87.64404 41.93558),6
666,1436495105198659328,W Armitage Ave & N Sheffield Ave,8,8,In Service,41.917805,-87.653449,"{'latitude': '41.917805', 'longitude': '-87.65...",16.0,21190,68.0,168.0,34.0,POINT (-87.65345 41.91781),7
667,1448642162257565184,S Wentworth Ave & W 111th St,5,5,In Service,41.692618,-87.628052,"{'latitude': '41.6926176', 'longitude': '-87.6...",45.0,21861,45.0,524.0,22.0,POINT (-87.62805 41.69262),49


In [18]:
divvy_2019_trips_by_chicago_area = divvy_2019.groupby('from_chicago_area').agg({'trip_id':'size', 'from_station_id':'nunique'})
divvy_2019_trips_by_chicago_area

,trip_id,from_station_id
from_chicago_area,,
Central,70410,35
Far North,93164,66
Far Southeast,26119,12
North,50301,30
Northwest,13644,23
South,1166805,138
Southwest,588404,128
West,719481,163


KeyError: "None of [Index(['South', 'West', 'Central', 'Southwest', 'Far North', 'Far Southeast',\n       'North', 'Northwest'],\n      dtype='object')] are in the [columns]"